In [1]:
import requests
from tqdm import tqdm

url_table_logement = "https://www.data.gouv.fr/fr/datasets/r/f314175a-6d33-4ee4-b5eb-2cb6c29df2c2"
url_table_individu = "https://www.data.gouv.fr/fr/datasets/r/c8e1b241-75fe-43e9-a266-830fc30ec61d"
url_doc_logement = "https://www.data.gouv.fr/fr/datasets/r/c274705f-98db-4d9b-9674-578e04f03198"
url_doc_individu = "https://www.data.gouv.fr/fr/datasets/r/1c6c6ab2-b766-41a4-90f0-043173d5e9d1"
url_bpe = "https://www.insee.fr/fr/statistiques/fichier/8217525/BPE23.parquet"

def download_file(url: str, filename: str) -> None:
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)
        # Raise an exception for HTTP errors
        response.raise_for_status()
        
        # Get the total file size from the headers (if available)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024  # 1 Kilobyte

        # Progress bar setup
        progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True, desc=filename)
        
        # Write the content to the file
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=block_size):
                progress_bar.update(len(chunk))
                file.write(chunk)
        
        progress_bar.close()
        print(f"File downloaded successfully: {filename}")
    
    except requests.exceptions.RequestException as e:
        print(f"Failed to download the file: {e}")


download_file(url_table_logement, "RPlogement.parquet")
download_file(url_table_individu, "RPindividus.parquet")
download_file(url_doc_logement, "RPlogement_doc.csv")
download_file(url_doc_individu, "RPindividus_doc.csv")
download_file(url_bpe, "BPE23.parquet")

RPlogement.parquet: 100%|██████████| 504M/504M [00:23<00:00, 21.7MiB/s] 


File downloaded successfully: RPlogement.parquet


RPindividus.parquet: 100%|██████████| 533M/533M [00:23<00:00, 22.6MiB/s] 


File downloaded successfully: RPindividus.parquet


RPlogement_doc.csv: 36.4kiB [00:00, 5.60MiB/s]


File downloaded successfully: RPlogement_doc.csv


RPindividus_doc.csv: 48.6kiB [00:00, 26.6MiB/s]


File downloaded successfully: RPindividus_doc.csv


BPE23.parquet: 172MiB [00:56, 3.05MiB/s] 

File downloaded successfully: BPE23.parquet


In [10]:
import duckdb

# établissements d'enseignement
# Dico variable https://www.insee.fr/fr/statistiques/8217525?sommaire=8217537#consulter

bpe = duckdb.sql(
"""
SELECT * 
FROM read_parquet('BPE23.parquet') 
WHERE DEPCOM = '93001' 
  AND starts_with(TYPEQU, 'C')
  AND NOT (starts_with(TYPEQU, 'C6') OR starts_with(TYPEQU, 'C7'))
"""
).to_df()

bpe_complete = duckdb.sql(
"""
SELECT * 
FROM read_parquet('BPE23.parquet') 
"""
).to_df()


In [14]:
import geopandas as gpd
bpe_gdf = gpd.GeoDataFrame(
    bpe_complete, geometry=gpd.points_from_xy(bpe_complete.LONGITUDE, bpe_complete.LATITUDE), crs="EPSG:4326"
)
bpe_gdf.to_parquet("test.parquet", write_covering_bbox=True)

In [23]:
df = duckdb.sql(
    """
        SELECT *
        FROM read_parquet('test.parquet')
        WHERE
            starts_with(TYPEQU, 'F1')
            AND bbox.xmin BETWEEN 2.327943 AND 2.399054
            AND bbox.ymin BETWEEN 48.912967 AND 48.939077
    """
).to_df()


In [25]:
df.columns

Index(['AN', 'NOMRS', 'CNOMRS', 'NUMVOIE', 'INDREP', 'TYPVOIE', 'LIBVOIE',
       'CADR', 'CODPOS', 'DEPCOM', 'DEP', 'REG', 'DOM', 'SDOM', 'TYPEQU',
       'SIRET', 'STATUT_DIFFUSION', 'CANTINE', 'INTERNAT', 'RPI', 'EP',
       'CL_PGE', 'SECT', 'ACCES_AIRE_PRATIQUE', 'ACCES_LIBRE',
       'ACCES_SANITAIRE', 'ACCES_VESTIAIRE', 'CAPACITE_D_ACCUEIL',
       'PRES_DOUCHE', 'PRES_SANITAIRE', 'SAISONNIER', 'COUVERT', 'ECLAIRE',
       'CATEGORIE', 'MULTIPLEXE', 'ACCUEIL', 'ITINERANCE', 'MODE_GESTION',
       'SSTYPHEB', 'TYPE', 'CAPACITE', 'INDIC_CAPA', 'NBEQUIDENT',
       'INDIC_NBEQUIDENT', 'NBSALLES', 'INDIC_NBSALLES', 'NBLIEUX',
       'INDIC_NBLIEUX', 'LAMBERT_X', 'LAMBERT_Y', 'LONGITUDE', 'LATITUDE',
       'QUALITE_XY', 'EPSG', 'QUALITE_GEOLOC', 'TR_DIST_PRECISION', 'DCIRIS',
       'QUALI_IRIS', 'IRISEE', 'QP2015', 'QUALI_QP2015', 'QP', 'QUALI_QP',
       'QVA', 'QUALI_QVA', 'ZUS', 'QUALI_ZUS', 'EPCI', 'UU2020', 'BV2022',
       'AAV2020', 'DENS3', 'DENS7', 'LIBCOM', 'geometry', 'b

In [35]:
toto = duckdb.sql(
    """
    WITH parquet_data AS (
        SELECT *
        FROM read_parquet('test.parquet')
        WHERE
            starts_with(TYPEQU, 'F1')
            AND bbox.xmin BETWEEN 2.327943 AND 2.399054
            AND bbox.ymin BETWEEN 48.912967 AND 48.939077
    )
    SELECT parquet_data.*, csv_data.Libelle_TYPEQU
    FROM parquet_data
    JOIN read_csv_auto('https://www.insee.fr/fr/metadonnees/source/fichier/BPE23_table_passage.csv') AS csv_data
    ON parquet_data.TYPEQU = csv_data.TYPEQU
    """
).to_df()

df = gpd.GeoDataFrame(
    toto.drop("geometry", axis = "columns"), geometry=gpd.points_from_xy(toto.LONGITUDE, toto.LATITUDE), crs="EPSG:4326"
)

In [73]:
print(duckdb.sql(
    """
        EXPLAIN ANALYZE
        SELECT *
        FROM read_parquet('https://www.data.gouv.fr/fr/datasets/r/f314175a-6d33-4ee4-b5eb-2cb6c29df2c2')
    """
).to_df()['explain_value'].iloc[0]
)

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
         EXPLAIN ANALYZE         SELECT *         FROM read_parquet('https://www.data.gouv.fr/fr/datasets/r/f314175a-6d33-4ee4-b5eb-2cb6c29df2c2')     
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││            HTTP Stats:            ││
││                                   ││
││           in: 481.0 MiB           ││
││            out: 0 bytes           ││
││              #HEAD: 1             ││
││              #GET: 27             ││
││              #PUT: 0              ││
││              #POST: 0             ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││         Total Time: 5.80s         ││
│└───────────────────────────────────┘│
└───────

In [75]:
print(duckdb.sql(
    """
        EXPLAIN ANALYZE
        SELECT *
        FROM read_parquet('https://www.data.gouv.fr/fr/datasets/r/f314175a-6d33-4ee4-b5eb-2cb6c29df2c2')
        WHERE COMMUNE = '06088' and AEMM > 2020
    """
).to_df()['explain_value'].iloc[0]
)

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
         EXPLAIN ANALYZE         SELECT *         FROM read_parquet('https://www.data.gouv.fr/fr/datasets/r/f314175a-6d33-4ee4-b5eb-2cb6c29df2c2')         WHERE COMMUNE = '06088' and AEMM > 2020     
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││            HTTP Stats:            ││
││                                   ││
││            in: 19.8 MiB           ││
││            out: 0 bytes           ││
││              #HEAD: 1             ││
││              #GET: 73             ││
││              #PUT: 0              ││
││              #POST: 0             ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││         Total Time: 7.15s         ││


In [82]:
print(duckdb.sql(
    """
        EXPLAIN ANALYZE
        SELECT CAST(
            SUM(IPONDL*CAST(INPER AS INT)) AS INT
            ) AS habitants_toulouse_demenagement
            FROM read_parquet('https://www.data.gouv.fr/fr/datasets/r/f314175a-6d33-4ee4-b5eb-2cb6c29df2c2')
            WHERE COMMUNE == '31555' AND IRANM NOT IN ('1', 'Z') AND INPER != 'Y'
    """
).to_df()['explain_value'].iloc[0]
)

{
   "name":  "Query", 
   "result": 0.477482,
   "timing": 0.477601,
   "cardinality": 0,
   "extra-info": "\n        PRAGMA enable_profiling = 'json';\n        PRAGMA profiling_output = 'toto.json';\n        \n        EXPLAIN ANALYZE\n        SELECT CAST(\n            SUM(IPONDL*CAST(INPER AS INT)) AS INT\n            ) AS habitants_toulouse_demenagement\n            FROM read_parquet('https://www.data.gouv.fr/fr/datasets/r/f314175a-6d33-4ee4-b5eb-2cb6c29df2c2')\n            WHERE COMMUNE == '31555' AND IRANM NOT IN ('1', 'Z') AND INPER != 'Y'\n    ", 
   "timings": [

   ],
   "children": [
    {
      "name": "RESULT_COLLECTOR",
      "timing":0.000000,
      "cardinality":0,
      "extra_info": "",
      "children": [
       {
         "name": "EXPLAIN_ANALYZE",
         "timing":0.000000,
         "cardinality":0,
         "extra_info": "",
         "children": [
          {
            "name": "PROJECTION",
            "timing":0.000007,
            "cardinality":1,
            

In [80]:
print(duckdb.sql(
    """
        EXPLAIN ANALYZE
        SELECT *
        FROM read_parquet('test.parquet')
        WHERE
            starts_with(TYPEQU, 'F1')
            AND bbox.xmin BETWEEN 2.327943 AND 2.399054
            AND bbox.ymin BETWEEN 48.912967 AND 48.939077
    """
).to_df()['explain_value'].iloc[0]
)

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
         EXPLAIN ANALYZE         SELECT *         FROM read_parquet('test.parquet')         WHERE             starts_with(TYPEQU, 'F1')             AND bbox.xmin BETWEEN 2.327943 AND 2.399054             AND bbox.ymin BETWEEN 48.912967 AND 48.939077     
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││         Total Time: 0.389s        ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌───────────────────────────┐
│      RESULT_COLLECTOR     │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│      EXPLAIN_ANALYZE      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00

In [66]:
import folium
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex

df['Libelle_TYPEQU'] = df['Libelle_TYPEQU'].str[0] + df['Libelle_TYPEQU'].str[1:].str.lower()
df['NOMRS'] = df['NOMRS'].str[0] + df['NOMRS'].str[1:].str.lower()

# Create a base map centered around the average of the coordinates provided
map_center = [df['geometry'].y.mean(), df['geometry'].x.mean()]
folium_map = folium.Map(location=map_center, zoom_start=14)

# Iterate through the DataFrame and add markers
for _, row in df.iterrows():
    # Extract latitude and longitude from geometry
    lat, lon = row['geometry'].y, row['geometry'].x
    # Extract label and type for the popup and color
    label = row['Libelle_TYPEQU']
    name = row['NOMRS']
    typequ = row['TYPEQU']

    # Add a marker to the map
    folium.Marker(
        location=[lat, lon],
        popup=f"<b>{name}</b>\n \n \n<b>Catégorie</b>: {label} ({typequ})",  # Use NOMRS, Libelle_TYPEQU, and TYPEQU as the popup information
        icon=folium.Icon(icon="info-sign"),
    ).add_to(folium_map)

# Display the map
# folium_map